In [1]:
%matplotlib qt
import hyperspy.api as hs
import numpy
import matplotlib.pyplot as plt
import csv
plt.rcParams.update({'font.size': 20})
from hyperspy.misc.eels.tools import get_edges_near_energy

# Non-sparse Hyperspectrum

In [14]:
#file = numpy.loadtxt("1_0_1024_spimComplete", delimiter = ',', dtype=int)

ss = 32
how_many = 232
roll = 0

for slices in range(how_many):
    with open("results/"+str(how_many)+"_"+str(ss)+"_"+str(ss)+"_SpimComplete"+str(slices), "r") as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            full_array = numpy.asarray(row, dtype=float)
            full_array = numpy.reshape(full_array, (ss, ss, 1025))
            full_array = numpy.roll(full_array, roll, axis=0)
            s = hs.signals.Signal1D(full_array)
            s.save(f"results/slices/spim{slices}", overwrite=True)

In [15]:
s = hs.load("results/slices/*", stack=True)
s.set_signal_type("EELS")

[########################################] | 100% Completed |  0.1s


In [17]:
s.sum().plot()

# Sparse Hyperspectrum

In [5]:
#file = numpy.loadtxt("1_0_1024_spimComplete", delimiter = ',', dtype=int)

ss = 32
how_many = 232
roll = 0



for slices in range(how_many):
    f1 = open("results/232/"+str(how_many)+"_"+str(ss)+"_"+str(ss)+"_SparseSpimComplete"+str(slices), "r")
    r1 = csv.reader(f1, delimiter=',')
    
    f2 = open("results/232/"+str(how_many)+"_"+str(ss)+"_"+str(ss)+"_SparseSpimComplete_Hits"+str(slices), "r")
    r2 = csv.reader(f2, delimiter=',')
    
    for i in r1:
        for h in r2:
            spec = numpy.zeros(ss*ss*1025)
            i = numpy.asarray(i, dtype=int)
            h = numpy.asarray(h, dtype=int)
            spec[i] = h
            spec = numpy.reshape(spec, (ss, ss, 1025))
            s = hs.signals.Signal1D(spec)
            s.save(f"results/slices/spim{slices}", overwrite=True)

In [6]:
s = hs.load("results/slices/*", stack=True)
s.set_signal_type("EELS")

[########################################] | 100% Completed |  0.1s


In [7]:
s.plot()

# Sparse Hyperspectrum v2

## Summed spectral image (no temporal resolution)

In [2]:
#Choose your parameters
ss = 46
channels = 1041

#Do not touch
size = 100000000
n = 0;
total_read = 0
si = numpy.zeros(ss * ss * channels, dtype='int32')

while True:
    file = numpy.fromfile("si_complete.txt", count = size, offset = 4 * size * n, dtype='uint32')
    total_read = total_read + 4 * size
    print(f'Total bytes read (MB) is {total_read/1e6}.')
    if (file.shape[0] == 0): break
    unique, counts = numpy.unique(file, return_counts=True)
    si[unique] += counts
    n+=1
    
si = numpy.reshape(si, (ss, ss, channels))
s = hs.signals.Signal1D(si)
s.plot()

Total bytes read (MB) is 400.0.
Total bytes read (MB) is 800.0.
Total bytes read (MB) is 1200.0.
Total bytes read (MB) is 1600.0.
Total bytes read (MB) is 2000.0.
Total bytes read (MB) is 2400.0.
Total bytes read (MB) is 2800.0.
Total bytes read (MB) is 3200.0.
Total bytes read (MB) is 3600.0.
Total bytes read (MB) is 4000.0.
Total bytes read (MB) is 4400.0.
Total bytes read (MB) is 4800.0.
Total bytes read (MB) is 5200.0.
Total bytes read (MB) is 5600.0.
Total bytes read (MB) is 6000.0.
Total bytes read (MB) is 6400.0.
Total bytes read (MB) is 6800.0.
Total bytes read (MB) is 7200.0.
Total bytes read (MB) is 7600.0.
Total bytes read (MB) is 8000.0.
Total bytes read (MB) is 8400.0.
Total bytes read (MB) is 8800.0.
Total bytes read (MB) is 9200.0.
Total bytes read (MB) is 9600.0.
Total bytes read (MB) is 10000.0.
Total bytes read (MB) is 10400.0.
Total bytes read (MB) is 10800.0.
Total bytes read (MB) is 11200.0.
Total bytes read (MB) is 11600.0.
Total bytes read (MB) is 12000.0.
Total 

## Hyperspectral image with temporal resolution

In [3]:
#Do not touch. This will tell you how many frames you have
n = 0
size = 100000000
max_frames = 0

while True:
    file2 = numpy.fromfile("si_complete_indices.txt", count = size, offset = 2 * size * n, dtype='uint16')
    if (file2.shape[0] == 0): break
    temp = int(numpy.max(file2)) + 1
    if temp > max_frames: max_frames = temp
    n+=1
    
print(f"Maximum number of frames is {max_frames}.")

Maximum number of frames is 16973.


In [4]:
#Choose the frame bin and the scan parameters
frame_bin = 10
ss = 46
channels = 1041

#Do not touch
n = 0
total_read = 0
frames = int(max_frames/frame_bin+1)
print(f'Total number of frames will be {frames}.')

si = numpy.zeros(ss * ss * frames * channels, dtype='int16')

while True:
    file = numpy.fromfile("si_complete.txt", count = size, offset = 4 * size * n, dtype='uint32')
    file2 = numpy.fromfile("si_complete_indices.txt", count = size, offset = 2 * size * n, dtype='uint16')
    if (file2.shape[0] == 0): break
    total_read = total_read + 4 * size
    print(f'Total bytes read (MB) is {total_read/1e6}.')
    file2 = numpy.divide(file2, frame_bin)
    net_index = numpy.add(file, file2.astype('uint64') * ss * ss * channels, dtype='uint64')
    unique, counts = numpy.unique(net_index, return_counts=True)
    si[unique] += counts
    n+=1

si = numpy.reshape(si, (frames, ss, ss, channels))
s = hs.signals.Signal1D(si)
s

Total number of frames will be 1698.
Total bytes read (MB) is 400.0.
Total bytes read (MB) is 800.0.
Total bytes read (MB) is 1200.0.
Total bytes read (MB) is 1600.0.
Total bytes read (MB) is 2000.0.
Total bytes read (MB) is 2400.0.
Total bytes read (MB) is 2800.0.
Total bytes read (MB) is 3200.0.
Total bytes read (MB) is 3600.0.
Total bytes read (MB) is 4000.0.
Total bytes read (MB) is 4400.0.
Total bytes read (MB) is 4800.0.
Total bytes read (MB) is 5200.0.
Total bytes read (MB) is 5600.0.
Total bytes read (MB) is 6000.0.
Total bytes read (MB) is 6400.0.
Total bytes read (MB) is 6800.0.
Total bytes read (MB) is 7200.0.
Total bytes read (MB) is 7600.0.
Total bytes read (MB) is 8000.0.
Total bytes read (MB) is 8400.0.
Total bytes read (MB) is 8800.0.
Total bytes read (MB) is 9200.0.
Total bytes read (MB) is 9600.0.
Total bytes read (MB) is 10000.0.
Total bytes read (MB) is 10400.0.
Total bytes read (MB) is 10800.0.
Total bytes read (MB) is 11200.0.
Total bytes read (MB) is 11600.0.
Tot

<Signal1D, title: , dimensions: (46, 46, 1698|1041)>

In [5]:
s.inav[:, :, :].sum(axis=2).plot()
s.inav[:, :, 0].plot()
s.inav[:, :, 1].plot()